In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob

In [49]:
df_in = pd.read_csv('../final_data_files/uncrowded_photometry_bv.csv')
df_new = pd.DataFrame(data=np.zeros((1,17)), columns=df_in.columns)
print df_new

def get_data(line):
    mag = line.split('> = ')[1].split(' ')[0]
    merr = line.split('dev = ')[1].split(' ')[0]
    if mag == 'nan':
        mag = np.nan
    else:
        mag = float(mag)
    if merr == 'nan':
        merr = np.nan
    else:
        merr == float(merr)
    return mag, merr

kaluz = pd.read_csv('../Kaluzny2004rrl.txt', delim_whitespace=True)

def parse_gloess_output(gloessfile, df_new):
    starid = int(gloessfile.split('/')[1].split('.')[0])
    nanarray = np.zeros((1,17))
    nanarray[nanarray == 0] = np.nan
    new_row = pd.DataFrame(data = nanarray, columns=df_in.columns)
    new_row.id = starid
    new_row.per = kaluz.per[kaluz.id == starid].values[0]
    new_row.type = kaluz.type[kaluz.id == starid].values[0]
    f = open(gloessfile)
    for line in f:
        mag, merr = get_data(line)
        if line.startswith('<B'):
            new_row.mag_b = mag
            new_row.merr_b = merr
        if line.startswith('<V'):
            new_row.mag_v = mag
            new_row.merr_v = merr
        if line.startswith('<J'):
            new_row.mag_j = mag
            new_row.merr_j = merr
        if line.startswith('<H'):
            new_row.mag_h = mag
            new_row.merr_h = merr
        if line.startswith('<K'):
            new_row.mag_k = mag
            new_row.merr_k = merr
        if line.startswith('<[3'):
            new_row.mag_3 = mag
            new_row.merr_3 = merr
        if line.startswith('<[4'):
            new_row.mag_4 = mag
            new_row.merr_4 = merr
    f.close()
    df_new = pd.concat([df_new, new_row], ignore_index=True)
    return df_new

   id  mag_j  merr_j  type  per  mag_h  merr_h  mag_k  merr_k  mag_3  merr_3  \
0   0      0       0     0    0      0       0      0       0      0       0   

   mag_4  merr_4  mag_v  merr_v  mag_b  merr_b  
0      0       0      0       0      0       0  

[1 rows x 17 columns]


In [51]:
for i in glob.glob('multiband_output/*.glo_avs'):
    df_new = parse_gloess_output(i, df_new)

In [60]:
df_new = df_new.drop(df_new[df_new.id == 0].index).sort(columns='id')

In [62]:
df_new.to_csv('../final_data_files/photometry_gloess.csv', index=False)

In [78]:
df_out = df_new.merge(df_in, how='left', on=[i for i in df_new.columns])

In [79]:
df_out.to_csv('../final_data_files/photometry_gloess_samestars.csv', index=False)